In [189]:
import networkx as nx
import ndlib.models.ModelConfig as mc
import ndlib.models.epidemics as ep
from ndlib.viz.mpl.DiffusionTrend import DiffusionTrend
import matplotlib.pyplot as plt
import numpy as np
import csv

In [242]:


### CITE NETWORKX AND NDLIB !!!!!
def give_mean(data):
    return np.mean(list(data.values()))

def give_properties(G):
    # Model properties
    clustering_coefficient = give_mean(nx.clustering(G, nodes=None, weight=None))
    closeness_centrality = give_mean(nx.closeness_centrality(G, u=None, distance=None, wf_improved=True))
    betweenness_centrality = give_mean(nx.betweenness_centrality(G, k=None, normalized=True, weight=None, endpoints=False, seed=None))
    degree_centrality = give_mean(nx.degree_centrality(G))
    average_shortest_path_length = nx.average_shortest_path_length(G, weight=None, method=None)

    properties = np.matrix([clustering_coefficient, closeness_centrality, betweenness_centrality, degree_centrality, average_shortest_path_length])

    return(properties)

def evaluate(beta, gamma, I, n_iterations, G, export_name="untitled.pdf", vis=False):
    # Model selection
    model = ep.SIRModel(G)

    # Model Configuration
    cfg = mc.Configuration()
    cfg.add_model_parameter('beta', beta)
    cfg.add_model_parameter('gamma', gamma)
    cfg.add_model_parameter("fraction_infected", I)

    model.set_initial_status(cfg)

    # Simulation execution
    iterations = model.iteration_bunch(n_iterations)
    trends = model.build_trends(iterations)

    # Visualization
    if vis == True:
        viz = DiffusionTrend(model, trends)
        viz.plot(export_name)

    return trends

In [191]:
# Generate Networks of equivalent Form
n = 100
p = 0.1
k = int(n * p)
m = int(n * p)

runs=10
properties_er, properties_ws, properties_ba = np.zeros((5,runs)), np.zeros((5,runs)), np.zeros((5,runs))
er_list, ws_list, ba_list = [], [], []

for i in range(0, runs):
    seed=int(i)
    er = nx.erdos_renyi_graph(n, p, seed)   
    properties_er[:,i] = give_properties(er)
    er_list.append(er)
    
print('done1')
for i in range(0, runs):
    seed=int(i)
    ws = nx.watts_strogatz_graph(n, k, p, seed)
    properties_ws[:,i] = give_properties(ws)
    ws_list.append(ws)

print('done2')
for i in range(0, runs):
    seed=int(i)
    ba = nx.barabasi_albert_graph(n, m, seed)
    properties_ba[:,i] = give_properties(ba)
    ba_list.append(ba)

done1
done2


In [192]:
properties_string = ["clustering_coefficient", "closeness_centrality", "betweenness_centrality", "degree_centrality", "average_shortest_path_length"]

plt.hist(properties_er[0,:])
plt.savefig('test.png')

In [193]:
# Simulate SIR spread on the network
plt.clf()
beta_list =  [1, 1/3, 1/6] 
gamma = 1/3
I0_list = [0.001, 0.01, 0.1]
n_iterations = 50

for I0 in I0_list:
    for beta in beta_list:
        er_X, er_Y, er_Z = [0]*n_iterations, [0]*n_iterations, [0]*n_iterations
        ws_X, ws_Y, ws_Z = [0]*n_iterations, [0]*n_iterations, [0]*n_iterations
        ba_X, ba_Y, ba_Z = [0]*n_iterations, [0]*n_iterations, [0]*n_iterations

        for i in range(len(er_list)):
            er = er_list[i]
            ws = ws_list[i]
            ba = ba_list[i]

            trends_er = evaluate(beta, gamma, I0, n_iterations, er)
            trends_ws = evaluate(beta, gamma, I0, n_iterations, ws)
            trends_ba = evaluate(beta, gamma, I0, n_iterations, ba)

            er_X = np.add(er_X, trends_er[0]['trends']['node_count'][0])
            er_Y = np.add(er_Y, trends_er[0]['trends']['node_count'][1])
            er_Z = np.add(er_Z, trends_er[0]['trends']['node_count'][2])

            ws_X = np.add(ws_X, trends_ws[0]['trends']['node_count'][0]) 
            ws_Y = np.add(ws_Y, trends_ws[0]['trends']['node_count'][1]) 
            ws_Z = np.add(ws_Z, trends_ws[0]['trends']['node_count'][2]) 

            ba_X = np.add(ba_X, trends_ba[0]['trends']['node_count'][0]) 
            ba_Y = np.add(ba_Y, trends_ba[0]['trends']['node_count'][1]) 
            ba_Z = np.add(ba_Z, trends_ba[0]['trends']['node_count'][2]) 

        plt.ylim(0,1)
        plt.plot(er_Y/(runs*n))
        plot_string = 'plots2/er_beta-' + str(np.round(beta,2)) + '_gamma-' + str(np.round(gamma,2)) + '_I0-' + str(I0) + '.png'
        plt.savefig(plot_string)
        plt.clf()

        plt.ylim(0,1)
        plt.plot(ws_Y/(runs*n))
        plot_string = 'plots2/ws_beta-' + str(np.round(beta,2)) + '_gamma-' + str(np.round(gamma,2)) + '_I0-' + str(I0) + '.png'
        plt.savefig(plot_string)
        plt.clf()

        plt.ylim(0,1)
        plt.plot(ba_Y/(runs*n))
        plot_string = 'plots2/ba_beta-' + str(np.round(beta,2)) + '_gamma-' + str(np.round(gamma,2)) + '_I0-' + str(I0) + '.png'
        plt.savefig(plot_string)
        plt.clf()

/Users/jennadevries/anaconda3/lib/python3.10/site-packages/ndlib/models/DiffusionModel.py:169: UserWarning: The fraction_infected value is too low given the number of nodes of the selected graph: a single node will be set as infected
  warnings.warn(


In [246]:
# Dynamic vaccination campaign
# https://www.includehelp.com/python/load-csv-into-2d-matrix-with-numpy-for-plotting.aspx#:~:text=Loading%20CSV%20into%202D%20matrix,using%20the%20numpy%20array%20object.

"Creating network"
data = np.loadtxt(open("transmission_network.csv", "rb"), delimiter=";", skiprows=1)[:, 1:]
nonzero_x = np.nonzero(data)[0]
nonzero_y = np.nonzero(data)[1]

edges=[]
for i in range(len(nonzero_x)):
    edges.append((nonzero_x[i], nonzero_y[i]))

G = nx.from_edgelist(edges)

"Set parameters"
beta_list =  1
gamma = 1/3
I0 = 5/374
n_iterations = 50

X, Y, Z = [0]*n_iterations, [0]*n_iterations, [0]*n_iterations

trends = evaluate(beta, gamma, I0, n_iterations, G)

X = np.add(er_X, trends_er[0]['trends']['node_count'][0])
Y = np.add(er_Y, trends_er[0]['trends']['node_count'][1])
Z = np.add(er_Z, trends_er[0]['trends']['node_count'][2])


plt.ylim(0,1)
plt.plot(er_Y/(runs*n))
plt.savefig('vaccination.png')
plt.clf()

# SET INFECTED NODES
    cfg.add_model_initial_configuration("Infected", infected_nodes)
